# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['TGT_OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
from timm.scheduler import CosineLRScheduler
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
PREMODEL_ROOT = config['IO_OPTION']['PREMODEL_ROOT']
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['TGT_OUTPUT_ROOT']
MODEL_DIR = OUTPUT_ROOT + '/models'

TB_DIR = OUTPUT_ROOT + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = OUTPUT_ROOT + '/pred'
SCORE_DIR = OUTPUT_ROOT + '/score'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)
os.makedirs(SCORE_DIR, exist_ok=True)

# copy config
# shutil.copy('./config.yaml', OUTPUT_ROOT)

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
percent = 0.1

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # increse tgt
    num_increse_tgt = int(len(dev_train_source_paths) * percent)
    increse_dev_train_target_paths = dev_train_target_paths * num_increse_tgt
    # concat
    dev_train_paths = increse_dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(add_dev_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # increse tgt
    num_increse_tgt = int(len(add_dev_source_paths) * percent)
    increse_add_dev_target_paths = add_dev_target_paths * num_increse_tgt
    # concat
    add_dev_paths = increse_add_dev_target_paths

    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3114, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [11]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [12]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    
    model = Model(n_classes=6)
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'] + f'/{machine_type}_model.pkl')
    model.load_state_dict(pretrained_dict, strict=False)
    
    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc_block.parameters():
        param.requires_grad = True
    for param in model.CenterLoss.parameters():
        param.requires_grad = True
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    scheduler = CosineLRScheduler(optimizer, t_initial=100, lr_min=1e-6, 
                                  warmup_t=10, warmup_lr_init=5e-5, warmup_prefix=True)
    output_dict, model, pred_df, score_df = modeler.run_training(model, dataloaders_dict, writer, optimizer, scheduler)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    score_out_path = f'{SCORE_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    score_df.to_csv(score_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

In [13]:
#run(machine_types[0], dev_paths)

In [ ]:
for machine_type in machine_types[0:]:
    run(machine_type, dev_paths)

2021-11-06 19:23:32,985 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-11-06 19:23:32,986 - 00_train.py - INFO - MAKE DATA_LOADER
2021-11-06 19:23:32,995 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 600/600 [01:57<00:00,  5.10it/s]
2021-11-06 19:27:36,520 - pytorch_modeler.py - INFO - epoch:1/100, tr_loss:0.778110, src_loss:0.536107, src_mean_auc:0.832567, tgt_loss:0.491058, tgt_mean_auc:0.627233, mean_auc:0.729900,


,AUC,pAUC
Source_0,0.817500,0.614211
Source_1,0.926400,0.783684
Source_2,0.753800,0.549474
Target_0,0.683700,0.588421
Target_1,0.564500,0.474211
Target_2,0.633500,0.564211
mean,0.729900,0.595702
h_mean,0.710737,0.582260


100%|██████████| 22/22 [00:08<00:00,  2.56it/s]
2021-11-06 19:27:45,292 - pytorch_modeler.py - INFO - epoch:2/100, tr_loss:0.397572, src_loss:0.536107, src_mean_auc:0.832567, tgt_loss:0.491058, tgt_mean_auc:0.627233, mean_auc:0.729900,


...

100%|██████████| 22/22 [00:08<00:00,  2.67it/s]
2021-11-06 21:06:02,575 - pytorch_modeler.py - INFO - epoch:86/100, tr_loss:0.214276, src_loss:0.523887, src_mean_auc:0.746260, tgt_loss:0.524785, tgt_mean_auc:0.767998, mean_auc:0.757129,


,AUC,pAUC
Source_0,0.777631,0.546944
Source_1,0.783008,0.610407
Source_2,0.678140,0.503772
Target_0,0.769097,0.505259
Target_1,0.753944,0.556440
Target_2,0.780952,0.531119
mean,0.757129,0.542324
h_mean,0.755213,0.540017


100%|██████████| 22/22 [00:08<00:00,  2.57it/s]
2021-11-06 21:06:11,413 - pytorch_modeler.py - INFO - epoch:87/100, tr_loss:0.211526, src_loss:0.523887, src_mean_auc:0.746260, tgt_loss:0.524785, tgt_mean_auc:0.767998, mean_auc:0.757129,


,AUC,pAUC
Source_0,0.777631,0.546944
Source_1,0.783008,0.610407
Source_2,0.678140,0.503772
Target_0,0.769097,0.505259
Target_1,0.753944,0.556440
Target_2,0.780952,0.531119
mean,0.757129,0.542324
h_mean,0.755213,0.540017


 18%|█▊        | 4/22 [00:02<00:11,  1.62it/s]

In [ ]:
#run(machine_types[0], dev_paths)

# run